## Récupération des données

In [1]:
import os
import urllib.request

if("training_labels.txt" not in os.listdir(".")):
    print("Training labels are not in the root directory. Downloading...")
    urllib.request.urlretrieve("https://dl.dropboxusercontent.com/s/l0f9z08rysp0kjy/training_labels.txt", "training_labels.txt")
    print("Done.")
if("training_templates.csv" not in os.listdir(".")):
    print("Training data is not in the root directory. Downloading... this may take a while...")
    urllib.request.urlretrieve("https://dl.dropboxusercontent.com/s/dqudxed82ljnxa8/training_templates.csv", "training_templates.csv")
    print("Done.")
if("testing_templates.csv" not in os.listdir(".")):
    print("Testing data are not in the root directory. Downloading... this may take a while...")
    urllib.request.urlretrieve("https://dl.dropboxusercontent.com/s/syrry7miykrmjz0/testing_templates.csv", "testing_templates.csv")
    print("Done.")

import pandas as pd
import numpy as np


data = pd.read_csv("training_templates.csv", header=None)
X = np.array(data)
y = np.loadtxt("training_labels.txt")


In [ ]:
print("(Samples, features) = ",X.shape, sep=" ")
print(len(y))
print(np.unique(y))

(Samples, features) =  (105600, 128)
105600
[-1.  1.]


# Classification :

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

# Shuffling data
permut = np.random.permutation(len(X))
X,y = X[permut], y[permut]

# Scaling data:
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf = svm.SVC()
print("Fitting classifier...")
clf.fit(X_train, y_train)

# Performance
def compute_pred_score(y_true, y_pred):
    y_pred_unq =  np.unique(y_pred)
    for i in y_pred_unq:
        if((i != -1) & (i!= 1) & (i!= 0) ):
            raise ValueError('The predictions can contain only -1, 1, or 0!')
    y_comp = y_true * y_pred
    score = float(10*np.sum(y_comp == -1) + np.sum(y_comp == 0))
    score /= y_comp.shape[0]
    return score

print("Predicting test samples...")
y_pred = clf.predict(X_test)

s = compute_pred_score(y_test, y_pred)

print("Score on %d samples: %f" % len(y_pred), s)

Fitting classifier
